<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:100px; margin: 20px;" />
</div>

## Uploading Data into Exasol

We are going to perform some data engineering tasks on our dataset to creature features and upload them to the database for our model to have the best performance possible.
                                                                                                                       
## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI-Lab](../main_config.ipynb).

## Setup

### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

## Import dataset into pandas dataframe

From the csv file. We are importing the data into the Pandas Dataframe and converting the 'Order Date' and 'Ship Date' column to the pandas datetime format because that is the column on which we are going to predict on.

In [ ]:
from pandas import pandas as pd
from sklearn.preprocessing import LabelEncoder
data = pd.read_csv('stores_sales_forecasting.csv', encoding='ISO-8859-1')
data['Order Date'] = pd.to_datetime(data['Order Date'],errors='coerce')
data['Ship Date'] = pd.to_datetime(data['Ship Date'], errors='coerce')
data.head()

## Creating features for our XGBoost Model

The "create_features" function generates new features for a sales dataset, including time-based features (day, month, year, weekday) from 'Order Date' and 'Ship Date'. It creates lag features for sales (1-week and 1-month), calculates a 'quantity_discount_ratio', and one-hot encodes categorical variables. The function then removes irrelevant columns and handles missing values, returning the processed dataset for further analysis or modeling.

In [ ]:

def create_features_new(data,lags=[1,3,7,30]):
    """
    Creates time series features from datetime index
    """
    data['order_day'] = data['Order Date'].dt.day
    data['sales'] = data['Sales']
    data['order_month'] = data['Order Date'].dt.month
    data['order_year'] = data['Order Date'].dt.year
    data['order_weekday'] = data['Order Date'].dt.weekday
    data['ship_day'] = data['Ship Date'].dt.day
    data['ship_month'] = data['Ship Date'].dt.month
    data['ship_year'] = data['Ship Date'].dt.year
    data['ship_weekday'] = data['Ship Date'].dt.weekday
    categorical_columns = ['Segment', 'Country', 'City', 'Region', 'Category', 'Sub-Category', 'Ship Mode','State']
    data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
    data_encoded['sales_lag_1_week'] = data_encoded['Sales'].shift(7)
    data_encoded['sales_lag_1_month'] = data_encoded['Sales'].shift(30)
    data_encoded['quantity_discount_ratio'] = data_encoded['Quantity'] / (data_encoded['Discount'] + 1)
    data_encoded.dropna()
    
    return data_encoded, data_encoded.drop(columns=['Sales', 'Row ID', 'Order ID', 'Customer ID', 'Customer Name', 'Product Name', 'Product ID', 'Ship Date', 'Order Date']), data_encoded

Executing the function and storing the values to be uploaded to the database

In [ ]:

train_features, test_features, control_data =  create_features_new(data)

## Create tables and load the data into thise tables
Below we are creating two tables in the database "TRAIN_TABLE" and "TEST_TABLE" and uploading our corresponding feature sets into those tables.

In [ ]:
from exasol.nb_connector.connections import open_pyexasol_connection

dtype_map = {
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'object': 'VARCHAR(255)',
    'bool': 'BOOLEAN',
    'datetime64[ns]': 'TIMESTAMP'
}

train_table = "TRAIN_TABLE"
test_table = "TEST_TABLE"
control_table = "CONTROL_TABLE"
# Generating a CREATE TABLE statement
cols = []
for col_name, dtype in train_features.dtypes.items():
    sql_type = dtype_map.get(str(dtype), 'VARCHAR(255)')
    cols.append(f'"{col_name}" {sql_type}')

test_cols = []
for col_name, dtype in test_features.dtypes.items():
    sql_type = dtype_map.get(str(dtype), 'VARCHAR(255)')
    test_cols.append(f'"{col_name}" {sql_type}')

    
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    create_train_sql = f'CREATE OR REPLACE TABLE "{ai_lab_config.db_schema}"."{train_table}"({", ".join(cols)})'
    conn.execute(create_train_sql)
    create_test_sql = f'CREATE OR REPLACE TABLE "{ai_lab_config.db_schema}"."{test_table}"({", ".join(test_cols)})'
    conn.execute(create_test_sql)
  

    conn.import_from_pandas(train_features, (ai_lab_config.db_schema, train_table))
    print(f"Imported {conn.last_statement().rowcount()} rows into '{train_table}' table")
    conn.import_from_pandas(test_features, (ai_lab_config.db_schema, test_table))
    print(f"Imported {conn.last_statement().rowcount()} rows into '{test_table}' table")
    
    

We have uploaded our data now, we can move onto [creating and uploading a UDF into the database](./create_udf.ipynb) to execute the predict function on our test dataset.